## Load the training data into feature matrix, class labels, and event ids:

In [14]:
pri0_to_drop = ["DER_deltaeta_jet_jet","DER_mass_jet_jet","DER_prodeta_jet_jet","DER_lep_eta_centrality","PRI_jet_leading_pt","PRI_jet_leading_eta","PRI_jet_leading_phi","PRI_jet_subleading_pt","PRI_jet_subleading_eta","PRI_jet_subleading_phi"]
pri1_to_drop = ["DER_deltaeta_jet_jet","DER_mass_jet_jet","DER_prodeta_jet_jet","DER_lep_eta_centrality","PRI_jet_subleading_pt","PRI_jet_subleading_eta","PRI_jet_subleading_phi"]

In [2]:
# Useful starting lines
%matplotlib inline
import numpy as np
import numpy.linalg as la
import matplotlib.pyplot as plt
from expansion import *
%load_ext autoreload
%autoreload 2
import pandas as pd
from IPython.display import display
from proj1_helpers import *

In [7]:
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
hb = pd.read_csv(DATA_TRAIN_PATH, sep=',')
pd.options.display.max_columns = None

hb = hb.drop(['Id'], 1)
#hb.describe()

In [8]:
def cleanDataSet(dataset):
    dataset = dataset.replace(-999, np.nan)
    pri0 = dataset[dataset.PRI_jet_num==0].copy()
    pri0 = pri0.drop(pri0_to_drop,1)
    pri0 = pri0.drop(["PRI_jet_num","PRI_jet_all_pt"],1)

    pri1 = dataset[dataset.PRI_jet_num == 1].copy()
    pri1 = pri1.drop(pri1_to_drop,1)
    pri1 = pri1.drop(["PRI_jet_num"],1)

    pri2 = dataset[dataset.PRI_jet_num == 2].copy()
    pri2 = pri2.drop(["PRI_jet_num"],1)

    pri3 = dataset[dataset.PRI_jet_num == 3].copy()
    pri3 = pri3.drop(["PRI_jet_num"],1)
    
    return [pri0,pri1,pri2,pri3]

In [9]:
def extractPredictions(dataset):
    return dataset.Prediction.apply(lambda x: -1 if x == 'b' else 1)

In [10]:
def normalizeDataset(dataset):
    dataset = (dataset - dataset.mean()) / dataset.std()
    dataset = dataset.fillna(0)
    dataset = (dataset - dataset.mean()) / dataset.std()
    return dataset

In [12]:
def tildaNumpy(X):
    return np.c_[np.ones(X.shape[0]), X]

In [69]:
POLYNOMIAL_EXPANSION_DEGREE = 8

pri = cleanDataSet(hb)
predictions = []

for idx, dataset in enumerate(pri):
    predictions.append(extractPredictions(dataset))
    dataset = dataset.drop(['Prediction'],1)
    normalized_dataset = normalizeDataset(dataset)
    pri[idx] = tildaNumpy(polynomial_expansion( normalizeDataset(dataset).to_numpy(), POLYNOMIAL_EXPANSION_DEGREE))
    # pri[idx] is our tX depending on jet_num


*DATA cleaning tests*

In [23]:
test = np.array([[1,2,3],[4,5,6]])

In [25]:
test

array([[1, 2, 3],
       [4, 5, 6]])

In [54]:
polynomial_expansion(test,2)

array([[ 1,  2,  3,  1,  4,  9],
       [ 4,  5,  6, 16, 25, 36]])

In [30]:
d = {'col1': [1, 2], 'col2': [3, 4]}

In [31]:
df = pd.DataFrame(data=d)

In [32]:
df.to_numpy()

array([[1, 3],
       [2, 4]])

## Do your thing crazy machine learning thing here :) ...

In [40]:
def least_squares_GD(y, tx, initial_w, max_iters, gamma):
    """
    Linear regression using gradient descent.
    Uses MSE.
    
    Parameters
    ----------
    y:  ndarray
        the labels
    tx: ndarray
        vector x tilde, i.e. the parameters with a bias term
    initial_w: ndarray
        initial weight vector
    max_iters: int
        maximum number of iterations
    gamma: float
        learning rate

    Returns
    -------
    (ndarray, float)
        Last weight vector and the corresponding loss value
    """
    
    def compute_e(y, tx, w):
        return y - tx @ w
    
    def compute_loss(n2, e):
        return (e.T @ e) / n2
    
    def compute_gradient(tx, n, e):
        return - tx.T @ e / n
    
    loss = 0
    w = initial_w
    n = y.shape[0]
    n2 = n*2
    
    for n_iter in range(max_iters):
        e = compute_e(y, tx, w)
        gradient = compute_gradient(tx, n, e)
        loss = compute_loss(n2, e)
        
        # Update weights
        w -= gamma * gradient
        #print("Gradient Descent({bi}/{ti}): loss={l}, w={w}".format(
        #      bi=n_iter, ti=max_iters - 1, l=loss, w=w[0]))

    return w, loss

In [ ]:
'''
weights = np.array([])
for i in range(100):
    initial_w = np.full(tX.shape[1], i/100)
    max_iters = 100
    gamma = 0.3
    w, loss = least_squares_GD(y, tX, initial_w, max_iters, gamma)
    weights = np.append(weights, loss)
idx = np.argmin(weights)
'''
initial_w = np.zeros(tX.shape[1])
max_iters = 1000
gamma = 0.3
w, loss = least_squares_GD(y, tX, initial_w, max_iters, gamma)
print(loss)

In [ ]:
def least_squares_GD(y, tx, initial_w, max_iters, gamma):
    """
    Linear regression using gradient descent.
    Uses MAE.
    
    Parameters
    ----------
    y:  ndarray
        the labels
    tx: ndarray
        vector x tilde, i.e. the parameters with a bias term
    initial_w: ndarray
        initial weight vector
    max_iters: int
        maximum number of iterations
    gamma: float
        learning rate

    Returns
    -------
    (ndarray, float)
        Last weight vector and the corresponding loss value
    """
    
    def compute_e(y, tx, w):
        return y - tx @ w
    
    def compute_loss(n, e):
        return 1/n * np.sum(np.abs(e))
    
    def compute_gradient(tx, n, e):
        e = y - tx @ w
    
        return -1/n*tx.T @ np.sign(e)
    
    loss = 0
    w = initial_w
    n = y.shape[0]
    n2 = n*2
    
    for n_iter in range(max_iters):
        e = compute_e(y, tx, w)
        gradient = compute_gradient(tx, n, e)
        loss = compute_loss(n2, e)
        
        # Update weights
        w -= gamma * gradient
        #print("Gradient Descent({bi}/{ti}): loss={l}, w={w}".format(
        #      bi=n_iter, ti=max_iters - 1, l=loss, w=w[0]))

    return w, loss

In [ ]:
initial_w = np.zeros(tX.shape[1])
max_iters = 1000
gamma = 0.3
w, loss = least_squares_GD(y, tX, initial_w, max_iters, gamma)
print(loss)

In [59]:
def least_squares_SGD(y, tx, initial_w, max_iters, gamma):
    """
    Linear regression using stochastic gradient descent.
    Uses MAE.
    
    Parameters
    ----------
    y:  ndarray
        the labels
    tx: ndarray
        vector x tilde, i.e. the parameters with a bias term
    initial_w: ndarray
        initial weight vector
    max_iters: int
        maximum number of iterations
    gamma: float
        learning rate

    Returns
    -------
    (ndarray, float)
        Last weight vector and the corresponding loss value
    """
    
    def compute_e(y, tx, w):
        return y - tx @ w
    
    def compute_loss(n2, e):
        return (e.T @ e) / n2
    
    def compute_gradient(tx, n, e):
        return - tx.T @ e / n
    
    loss = 0
    w = initial_w[:, np.newaxis]
    n = y.shape[0]
    n2 = n*2
    data_size = len(y)
    shuffled_indices = np.random.permutation(np.arange(data_size))
    shuffled_y = y[shuffled_indices]
    shuffled_tx = tx[shuffled_indices]
    shuffled_y = shuffled_y[:,np.newaxis]
    for n_iter, by, btx in zip(range(max_iters), shuffled_y, shuffled_tx):
        by = by[np.newaxis]
        btx = btx[np.newaxis, :]
        e = compute_e(by, btx, w)
        gradient = compute_gradient(btx, n, e)
        loss = compute_loss(n2, e)
        
        # Update weights
        w -= gamma * gradient
        #print("Gradient Descent({bi}/{ti}): loss={l}, w={w}".format(
        #      bi=n_iter, ti=max_iters - 1, l=loss, w=w[0]))
    return w, compute_loss(n2, compute_e(y, tx, w[:,0]))

In [58]:
initial_w = np.full(tX.shape[1], 0.1)
max_iters = 100000
gamma = 0.7
w, loss = least_squares_SGD(y, tX, initial_w, max_iters, gamma)
print(loss)

NameError: name 'tX' is not defined

In [ ]:
def least_squares(y, tx):
    """
    Linear regression using normal equations.
    Use MSE loss function
    
    Parameters
    ----------
    y:  ndarray
        the labels
    tx: ndarray
        vector x tilde, i.e. the parameters with a bias term

    Returns
    -------
    (ndarray, float)
        Last weight vector and the corresponding loss value
    """    
    def compute_e(y, tx, w):
        return y - tx @ w
    
    def compute_loss(n2, e):
        return (e.T @ e) / n2
    
    w = la.solve(tx.T @ tx, tx.T @ y)
    
    return w, compute_loss(y.shape[0]*2, compute_e(y, tx, w))

w, loss = least_squares(y, tX)
print(loss)

In [ ]:
def ridge_regression(y, tx, lambda_):
    """
    Ridge regression using normal equations.
    
    Parameters
    ----------
    y : ndarray
        Description of y
    ...

    Returns
    -------
    (ndarray, float)
        Last weight vector and the corresponding loss value
    """ 
    
            
    def compute_e(y, tx, w):
        return y - tx @ w
    
    def compute_loss(n2, e):
        return (e.T @ e) / n2
    
    
    X = tx.T @ tx
    w = la.solve(X + lambda_ * (2*y.shape[0]) * np.eye(X.shape[0]), tx.T @ y)
    return w, compute_loss(y.shape[0]*2, compute_e(y, tx, w))



In [ ]:
def ridge_regression_demo(tx, y):

    
    """ridge regression demo."""
    # define parameter
    lambdas = np.logspace(-5, 0, 15)
    
    rmse_tr = []
    for ind, lambda_ in enumerate(lambdas):
        weights, loss = ridge_regression(y, tx, lambda_)
        rmse_tr.append(np.sqrt(loss))

        print("lambda={l:.3f}, Training RMSE={tr:.3f}".format(l=lambda_, tr=rmse_tr[ind]))
        

ridge_regression_demo(tX, y)

w, loss = ridge_regression(y, tX, 0.001)
print(loss)

In [ ]:
def sigmoid(t):
    """apply sigmoid function on t."""
    return 1/(1 + np.exp(-t))
    
def calculate_loss(y, tx, w):
    """compute the cost by negative log likelihood."""
    return np.sum(np.log(1 + np.exp(tx @ w)) - y * (tx @ w))


def calculate_gradient(y, tx, w):
    """compute the gradient of loss."""
    return tx.T @ (sigmoid(tx@w) - y)


def logistic_regression_step(y, tx, w):
    """return the loss, gradient"""
    return calculate_loss(y, tx, w), calculate_gradient(y, tx, w)
  

In [ ]:
# TODO
def logistic_regression(y, tx, lambda_):
    """
    Logistic regression using gradient descent or SGD.
    
    Parameters
    ----------
    y : ndarray
        Description of y
    ...

    Returns
    -------
    (ndarray, float)
        Last weight vector and the corresponding loss value
    """  
    def learning_by_gradient_descent(y, tx, w, gamma):
        """
        Do one step of gradient descen using logistic regression.
        Return the loss and the updated w.
        """
        loss = calculate_loss(y, tx, w)
        gradient = calculate_gradient(y,tx,w)
        w = w - gamma * gradient
        return loss, w
    
    # init parameters
    max_iter = 10000
    threshold = 1e-8
    #gamma = 0.01
    losses = []

    # build tx
    w = np.zeros((tx.shape[1], 1))
    y = y[:,np.newaxis]

    # start the logistic regression
    for iter in range(max_iter):
        # get loss and update w.
        loss, w = learning_by_gradient_descent(y, tx, w, lambda_)
        # log info
        if iter % 100 == 0:
            print("Current iteration={i}, loss={l}".format(i=iter, l=loss))
        # converge criterion
        losses.append(loss)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
     
    print("loss={l}".format(l=calculate_loss(y, tx, w)))
    return w, losses[-1]
    

In [ ]:
lambda_ = 1e-8
w, loss = logistic_regression(y, tX, lambda_)

In [ ]:
# TODO
def reg_logistic_regression(y, tx, lambda_):
    """
    Regularized logistic regression using gradient descent or SGD.
    
    Parameters
    ----------
    y : ndarray
        Description of y
    ...

    Returns
    -------
    (ndarray, float)
        Last weight vector and the corresponding loss value
    """    
    def penalized_logistic_regression(y, tx, w, lambda_):
        """return the loss, gradient"""
        loss, gradient = logistic_regression_step(y, tx, w)
        loss     += 2 * lambda_ * la.norm(w)**2
        gradient += lambda_ * w

        return loss, gradient
    
    def learning_by_penalized_gradient(y, tx, w, gamma, lambda_):
        """
        Do one step of gradient descent, using the penalized logistic regression.
        Return the loss and updated w.
        """
        loss, gradient = penalized_logistic_regression(y, tx, w, lambda_) 
        w = w - gamma * gradient 

        return loss, w
    
    # init parameters
    max_iter = 10000
    gamma = 1e-8
    threshold = 1e-8
    losses = []

    w = np.zeros((tx.shape[1], 1))
    y = y[:,np.newaxis]

    # start the logistic regression
    for iter in range(max_iter):
        # get loss and update w.
        loss, w = learning_by_penalized_gradient(y, tx, w, gamma, lambda_)
        # log info
        if iter % 100 == 0:
            print("Current iteration={i}, loss={l}".format(i=iter, l=loss))
        # converge criterion
        losses.append(loss)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
            
    print("loss={l}".format(l=calculate_loss(y, tx, w)))
    

In [ ]:
lambda_ = 0.000001
w, loss = reg_logistic_regression(y, tX, lambda_)

## End of ML magic

In [133]:
initial_w = np.full(pri[0].shape[1], 0.1)
max_iters = 6
gamma = 0.00000000000001
w, loss = least_squares_GD(predictions[0].to_numpy(), pri[0], initial_w, max_iters, gamma)
print(loss)

4.84627675481989e+198


In [118]:
for p in pri:
    print(np.isnan(p).sum())

0
0
0
0


In [117]:
for p in predictions:
    print(np.isnan(p.values).sum())

0
0
0
0


In [78]:
# methodes element of this contains [(methode_for_learning,(parameters_of_methode))]
pri_learn_func = [(least_squares_SGD, (np.zeros(pri[0].shape[1]), 3000, 0.00001)),
         (least_squares_SGD, (np.zeros(pri[1].shape[1]), 3000, 0.00001)),
         (least_squares_SGD,(np.zeros(pri[2].shape[1]), 3000, 0.00001)), 
         (least_squares_SGD, (np.zeros(pri[3].shape[1]), 3000, 0.00001))]




def learn(pri, pri_learn_func):
    w_pri = []
    for idx in range(len(pri)):
        learning_function, parameters = pri_learn_func[idx]
        w ,loss = learning_function(predictions[idx].to_numpy(),pri[idx],*parameters)
        print("* " + str(idx) + " loss : " + str(loss))
        w_pri.append(w)
    return w_pri

In [83]:
w_pri = learn(pri,pri_learn_func)

* 0 loss : nan
* 1 loss : nan
* 2 loss : nan
* 3 loss : nan


## Generate predictions and save ouput in csv format for submission:

In [ ]:
DATA_TEST_PATH = '../data/test.csv' # TODO: download train data and supply path here 
_, X_test, ids_test = load_csv_data(DATA_TEST_PATH)
hbt = pd.read_csv(DATA_TEST_PATH, sep=',')

#hbt = hbt.drop(['Id', 'Prediction'], 1)

In [ ]:
hbt = hbt.drop(['Prediction'], 1)

In [ ]:
hbt = hbt.set_index(['Id'])

In [ ]:
test_pri = cleanDataSet(hbt)
for idx , dataset in enumerate(test_pri):
    test_pri[idx] =  normalizeDataset(dataset)

In [ ]:
def predict(test_pri, w_pri):
    for idx, dataset in enumerate(test_pri):
        test_pri[idx]['Prediction'] = predict_labels(w_pri[idx],tildaNumpy(dataset))
    return test_pri

In [ ]:
test_prediction = predict(test_pri,w_pri)

In [ ]:
test_prediction[0]

In [ ]:
test_prediction[1]

In [ ]:
test_prediction[2]

In [ ]:
test_prediction[3]

In [ ]:
test_prediction = pd.concat(test_prediction,sort=True)

In [ ]:
test_prediction = test_prediction.sort_index()

In [ ]:
OUTPUT_PATH = 'predictions.csv'
create_csv_submission(test_prediction.Prediction.keys(), test_prediction.Prediction.values, OUTPUT_PATH)

In [ ]:
DATA_TEST_PATH = '../data/test.csv' # TODO: download train data and supply path here 
_, X_test, ids_test = load_csv_data(DATA_TEST_PATH)
hbt = pd.read_csv(DATA_TEST_PATH, sep=',')
hbt = hbt.drop(['Id', 'Prediction'], 1)
hbt = hbt.replace(-999, np.nan)
hbt = (hbt - hbt.mean()) / hbt.std()
hbt = hbt.fillna(0)
hbt = (hbt - hbt.mean()) / hbt.std()
tX_test = np.c_[np.ones(X_test.shape[0]), hbt.to_numpy()]

In [ ]:
OUTPUT_PATH = 'predictions.csv' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(w[1:], tX_test)#[:, [0, 1, 2, 3, 4, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]]) # Selected desired columns
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

In [ ]:
print(len(y_pred))
print(len(y_pred[y_pred > 0]))

# Submissions scores
Best score by technique

<ul>
    <li>MSE, gradient descent : 0.649</li>
    <li>MAE, gradient descent : 0.678 </li>
    <li>ridge regression      : 0.664</li>
</ul>
Best score after not being stupid with bias:

* MSE, GD: 
* MAE, GD: 0.639
* LSQ: 0.706
* R-REG: 0.730

Best score after normalizing test set + putting zero where unknown:

* LSQ: 0.747
* R-REG: 0.745

Feature expansion?